<a href="https://colab.research.google.com/github/iron59zug/YFapi/blob/master/api_iex_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# calling to API by a request w/ interpolated values
# IEX Cloud API not free of charge

import pandas as pd

In [11]:
a1 = 'MSFT' # 1975
a2 = 'APPL' # 1976
a3 = 'AMZN' # 1994
a4 = 'GOOGL' # 1998; Alphabet's A share has 1 vote; C share "GOOG" has no vote
a5 = 'FB' # 2004

ticker = a1 # choose one asset name that will pass to ticker, or:
tickers = [a1, a2, a3, a4, a5] # use a list of tickers
string = ','.join(tickers) # serialize list into string

endpoint = 'chart'
period = '1y'
KEY = '' # insert API key

request = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={string}&types={endpoint}&range={period}&token={KEY}'

In [16]:
# JavaScript Object Notation
df = pd.read_json(request)
df
# need API key to function!!!